Лабораторная работа 2. Провести предобработку данных. Разбить на тестовую и обучающую выборки (не забыть про кросс-валидацию).

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

import json
import csv

from prefs import prefs

INPUT_JSON = "data/full_data.json"
OUTPUT_CSV = "data/osm_nodes.csv"

Загрузка данных

In [86]:
with open(INPUT_JSON, "r", encoding="utf-8") as f:
    data = json.load(f)

nodes = [el for el in data.get("elements", []) if el.get("type") == "node"]

tag_keys = []
for node in nodes:

    x = node.get("tags", {}).get("tourism", {})
    if x == "yes":
        tag_keys.append("tourism")
    elif x != "no":
        tag_keys.append(node.get("tags", {}).get("tourism", {}))
    
    x = node.get("tags", {}).get("historic", {})
    if x == "yes":
        tag_keys.append("historic")
    elif x != "no":
        tag_keys.append(node.get("tags", {}).get("historic", {}))

    x = node.get("tags", {}).get("amenity", {})
    if x == "yes":
        tag_keys.append("amenity")
    elif x != "no":
        tag_keys.append(node.get("tags", {}).get("amenity", {}))

    x = node.get("tags", {}).get("leisure", {})
    if x == "yes":
        tag_keys.append("leisure")
    elif x != "no":
        tag_keys.append(node.get("tags", {}).get("leisure", {}))

tags = set()
for i in tag_keys:
    if i != {}:
        tags.add(i)

print("Всего тегов:",len(tags))
print("Теги:",tags)



Всего тегов: 79
Теги: {'historic', 'amusement_arcade', 'railway_station', 'place_of_worship', 'lean_to', 'propeller', 'aquarium', 'railway_car', 'apartment', 'fountain', 'cross', 'bench', 'manor', 'boundary_stone', 'garden', 'information', 'train', 'grave', 'clock', 'theme_park', 'archaeological_site', 'library', 'training', 'church', 'motel', 'gallery', 'aircraft', 'camp_site', 'restaurant', 'hotel', 'shield', 'caravan_site', 'anchor', 'museum', 'milestone', 'exhibit', 'memorial', 'sauna', 'mine', 'epigraph', 'city_gate', 'wayside_shrine', 'battlefield', 'locomotive', 'guest_house', 'tomb', 'picnic_table', 'attraction', 'trail_riding_station', 'binoculars', 'wayside_cross', 'missile', 'swimming_pool', 'monument', 'picnic_site', 'cafe', 'castle', 'artwork', 'viewpoint', 'zoo', 'suburb', 'tank', 'chalet', 'social_facility', 'printing_press', 'statue', 'water_park', 'ruins', 'ship', 'dormitory', 'stone', 'cannon', 'gate', 'wreck', 'vehicle', 'technical_monument', 'hostel', 'hospital', 'd

Преобразование в CSV

In [87]:
fieldnames = ["id", "lat", "lon", "name", "description", "inscription"] + sorted(tags)

with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()

    for node in nodes:
        row = {
            "id": node.get("id"),
            "lat": node.get("lat"),
            "lon": node.get("lon")
        }
        row["name"] = node.get("tags").get("name")
        row["description"] = node.get("tags").get("description")
        row["inscription"] = node.get("tags").get("inscription")
        

        if row["name"] == None:
            row["name"] = row["description"]

        if row["description"] == None:
            row["description"] = row["inscription"]

        if row["inscription"] == None:
            row["inscription"] = row["name"]

        if row["description"] == None:
            row["description"] = row["inscription"]
        
        if row["name"] == None:
            row["name"] = "Место-сюрприз"
        if row["description"] == None:
            row["description"] = "Место-сюрприз"
        if row["inscription"] == None:
            row["inscription"] = "Место-сюрприз"

        for key in tags:
            if(node.get("tags", {}).get(key, "")):
                row[key] = True
            else:
                row[key] = False
        writer.writerow(row)

print(f"Сохранено {len(nodes)} нод в {OUTPUT_CSV}")

Сохранено 7266 нод в data/osm_nodes.csv


In [90]:
df = pd.read_csv(OUTPUT_CSV, encoding="utf-8")

# df[binary_cols] = df[binary_cols].astype("int8")

df.info()
df.head()

# base_cols = {"id", "lat", "lon", "name", "description", "inscription"}
# binary_cols = [c for c in df.columns if c not in base_cols]
# df[binary_cols].sum().sort_values(ascending=False).head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7266 entries, 0 to 7265
Data columns (total 85 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    7266 non-null   int64  
 1   lat                   7266 non-null   float64
 2   lon                   7266 non-null   float64
 3   name                  7266 non-null   object 
 4   description           7266 non-null   object 
 5   inscription           7266 non-null   object 
 6   aircraft              7266 non-null   bool   
 7   amusement_arcade      7266 non-null   bool   
 8   anchor                7266 non-null   bool   
 9   apartment             7266 non-null   bool   
 10  aquarium              7266 non-null   bool   
 11  archaeological_site   7266 non-null   bool   
 12  artwork               7266 non-null   bool   
 13  attraction            7266 non-null   bool   
 14  battlefield           7266 non-null   bool   
 15  bench                

,id,lat,lon,name,description,inscription,aircraft,amusement_arcade,anchor,apartment,...,trail_riding_station,train,training,vehicle,viewpoint,water_park,wayside_cross,wayside_shrine,wreck,zoo
0,2,59.771793,30.326110,Центр круглого зала,Центр круглого зала,Центр круглого зала,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,238809,60.189557,29.700233,Башмаки неизвестного дачника (2009),Башмаки неизвестного дачника (2009),Башмаки неизвестного дачника (2009),False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,21626280,59.946264,30.395801,В. И. Ленин,В. И. Ленин,В. И. Ленин,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,21751966,60.039200,30.296405,Озёрная,Озёрная,Озёрная,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
4,80317281,59.970296,30.386136,М. И. Калинин,М. И. Калинин,М. И. Калинин,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [103]:
poi_cols = [col for col in prefs.keys() if col in df.columns]

poi_matrix = df[poi_cols]

weights = pd.Series([prefs[col] for col in poi_cols], index=poi_cols)

df['score'] = poi_matrix.dot(weights)

df[['id', 'name', 'score'] + poi_cols].head()

,id,name,score,historic,archaeological_site,ruins,battlefield,memorial,milestone,boundary_stone,...,cross,wayside_shrine,tomb,grave,attraction,picnic_site,picnic_table,trail_riding_station,suburb,sauna
0,2,Центр круглого зала,2,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,238809,Башмаки неизвестного дачника (2009),0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,21626280,В. И. Ленин,4,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,21751966,Озёрная,3,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,80317281,М. И. Калинин,4,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [108]:
df[['id', 'name', 'score']].sort_values('score', ascending=False).head(10)

,id,name,score
3507,7548851416,Город Пушкин,6
3489,7485870895,Часовня святого благоверного князя Игоря Черни...,6
5335,10915989243,Проспект Ю. Гагарина,6
343,1280066306,"ДОТ ""Миллионер""",6
243,1123474657,Памятник императору Николаю I,5
224,1041154862,Пётр I,5
6,242160221,Родина-Мать,5
8,249082841,Петру Великому,5
379,1312226165,Александру III,5
4517,9352981941,Место-сюрприз,4
